In [20]:
import os
import sys
import numpy as np
from pathlib import Path
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.initializers import Constant
from datetime import datetime


In [21]:
cur_dir = Path(os.getcwd())
print(cur_dir)
BASE_DIR = cur_dir
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'data')
MAX_SEQUENCE_LENGTH = 10000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


/Users/gkaufmann/Documents/GitHub/NN_final/NN_final


In [22]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
        
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [23]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

for name in sorted(os.listdir(TEXT_DATA_DIR)):
    print(name)
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname[:-4].isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

.DS_Store
Cannabis
LSD
MDMA
Mushrooms
Found 14825 texts.


In [24]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(data)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])


x_train = data[:(len(data)//10 * 8)]
y_train = labels[:(len(data)//10 * 8)]
x_val = data[(len(data)//10 * 8):]
y_val = labels[(len(data)//10 * 8):]




Found 32318 unique tokens.
[[    0     0     0 ...     8  6549  2770]
 [    0     0     0 ... 12731     2 12732]
 [    0     0     0 ...     4   445    55]
 ...
 [    0     0     0 ...   189   150    80]
 [    0     0     0 ...    47    27   201]
 [    0     0     0 ...   159    13   156]]
Shape of data tensor: (14825, 10000)
Shape of label tensor: (14825, 4)


In [25]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


Preparing embedding matrix.


In [26]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model.
Train on 11856 samples, validate on 2969 samples
Epoch 1/10
11856/11856 [==============================] - 580s 49ms/sample - loss: 1.2798 - acc: 0.4046 - val_loss: 1.1881 - val_acc: 0.4348
Epoch 2/10
11856/11856 [==============================] - 487s 41ms/sample - loss: 1.0870 - acc: 0.5155 - val_loss: 1.0624 - val_acc: 0.5204
Epoch 3/10
11856/11856 [==============================] - 473s 40ms/sample - loss: 0.9838 - acc: 0.5682 - val_loss: 1.0888 - val_acc: 0.5251
Epoch 4/10
11856/11856 [==============================] - 467s 39ms/sample - loss: 0.9202 - acc: 0.5987 - val_loss: 1.0034 - val_acc: 0.5537
Epoch 5/10
11856/11856 [==============================] - 477s 40ms/sample - loss: 0.8492 - acc: 0.6412 - val_loss: 1.0232 - val_acc: 0.5463
Epoch 6/10
11856/11856 [==============================] - 493s 42ms/sample - loss: 0.7642 - acc: 0.6800 - val_loss: 1.1894 - val_acc: 0.5338
Epoch 7/10
11856/11856 [==============================] - 477s 40ms/sample - loss: 0.6755

In [28]:
model.evaluate(x_train[:len(x_train)//10], y_train[:len(y_train)//10])

1185/1185 [==============================] - 16s 13ms/sample - loss: 0.3473 - acc: 0.8633


[0.3473097337071906, 0.86329114]

In [29]:
model.evaluate(x_val, y_val)

2969/2969 [==============================] - 40s 13ms/sample - loss: 1.4107 - acc: 0.5386


[1.4106536662590627, 0.53856516]

In [ ]:
# prepare generated data for evaluation by classification model

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(data)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])


x_train = data[:(len(data)//10 * 8)]
y_train = labels[:(len(data)//10 * 8)]
x_val = data[(len(data)//10 * 8):]
y_val = labels[(len(data)//10 * 8):]

In [9]:
def save_trained_model(trained_model, drug_name):
    now = datetime.now()
    model_json = trained_model.to_json()
    filename = drug_name + "_" + now.strftime("%m-%d_%H-%M-%S")
    with open("saved_models/" + filename + ".json", "w+") as json_file:
        json_file.write(model_json)
    # serialize weights
    trained_model.save_weights("saved_models/" + filename + ".h5")
    print(drug_name + "_" + now.strftime("%m-%d_%H-%M-%S") + " model saved!")

In [27]:
save_trained_model(model, 'classifier')

classifier_12-03_15-28-46 model saved!
